# 1 Введение

## 1.1 Постановка задачи

**Заказчик** -- оператор мобильной связи  
  
**Задача** -- построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра»

## 1.2 Исходные данные

### 1.2.1 Переданные файлы
- users_behavior.csv

### 1.2.2 Описание данных

#### 1.2.2.1 Таблица *users_behavior*
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- сalls — количество звонков;
- minutes — суммарная длительность звонков в минутах;
- messages — количество sms-сообщений;
- mb_used — израсходованный интернет-трафик в Мб;
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## 1.3 План обучения и оценки модели

### 1.3.1 Разделение набора на выборки
Из переданного набора данных выделить:
- обучающий набор данных;
- валидационный набор данных;
- тестовый набор данных.

### 1.3.2 Создание моделей
Поскольку по условиям задачи необходимо сформировать модель, предлагающую выбор из бинарного множества {0, 1}, создать модели:
- решающего дерева;
- случайного леса; 
- логистической регрессии.

### 1.3.3 Исследование качества моделей
Провести исследование качества полученных моделей на валидационной выборке, меняя значение гиперпараметров. Сделать выводы. Метрика качества -- *accuracy*.

### 1.3.4 Проверка настроенных моделей на тестовой выборке
Рассчитать *accuracy* моделей на тестовых данных.

### 1.3.5 Проверка моделей на вменяемость
Сравнить качество полученных моделей со случайной последовательностью ответов. Метрика качества -- *accuracy*. Сделать выводы об адекватности полученных моделей.

### 1.3.6 Вывод

## 1.4 Пользовательские функции

In [1]:
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

pd.options.display.float_format = '{:.2f}'.format

In [2]:
def shisda_predict(size=1, span=()):
    '''
    Фунция возвращает набор предиктных ответов модели "shot in the darkness" -- "пальцем в небо"
    '''

    random.seed()
    if not span:
        return [random.random() for i in range(size)] # случайные в диапазоне от 0 до 1
    if (type(span[0]) is int) and (type(span[1]) is int):
        return [random.randint(a=span[0], b=span[1]) for i in range(size)] # случайные целые в диапазоне span
    else:
        return [random.uniform(a=span[0], b=span[1]) for i in range(size)] # случайные с плавающей точкой в диапазоне span

# 2 Основная часть

## 2.1 Разделение набора на выборки

**Обзор данных**

In [3]:
users_behavior = pd.read_csv('users_behavior.csv')

In [4]:
users_behavior

,calls,minutes,messages,mb_used,is_ultra
0,40.00,311.90,83.00,19915.42,0
1,85.00,516.75,56.00,22696.96,0
2,77.00,467.66,86.00,21060.45,0
3,106.00,745.53,81.00,8437.39,1
4,66.00,418.74,1.00,14502.75,0
...,...,...,...,...,...
3209,122.00,910.98,20.00,35124.90,1
3210,25.00,190.36,0.00,3275.61,0
3211,97.00,634.44,70.00,13974.06,0
3212,64.00,462.32,90.00,31239.78,0


In [5]:
users_behavior.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Переданный набор данных предобработан, готов к использованию для формирования моделей машинного обучения.  
Целевой признак -- поле *is_ultra*.

**Разделение набора на выборки**

Выделим из переданного набора данных тренировочную, валидационную и тестовую выборки в соотношении 3:1:1 (0.6:0.2:0.2) соответственно. Для этого начальную выборку разделим на две выборки в соотнощении 0.6:0.4, а затем выборку 0.4 разделим на выборки в соотношении 0.5:0.5

In [6]:
df_train, df_valid = train_test_split(users_behavior, test_size=0.4, random_state=1456)
df_valid, df_test = train_test_split(df_valid, test_size=0.5, random_state=1456)

In [7]:
df_train.shape

(1928, 5)

In [8]:
df_valid.shape

(643, 5)

In [9]:
df_test.shape

(643, 5)

***Промежуточный итог***  
  
На данном этапе выполнения работы был произведён предварительный обзор данных, были выделены тренировочная, валидационная и тестовая выборки в соотношении 3:1:1 (0.6:0.2:0.2)

## 2.2 Создание моделей

In [10]:
train_features = df_train.drop(['is_ultra'], axis=1)
train_target = df_train['is_ultra']

**Решающее дерево**

In [11]:
from sklearn.tree import DecisionTreeClassifier

decision_tree_model = DecisionTreeClassifier(random_state=1456)
decision_tree_model.fit(train_features, train_target)

DecisionTreeClassifier(random_state=1456)

**Случайный лес**

In [12]:
from sklearn.ensemble import RandomForestClassifier

random_forest_model = RandomForestClassifier(random_state=1456)
random_forest_model.fit(train_features, train_target)

RandomForestClassifier(random_state=1456)

**Логистическая регрессия**

In [13]:
from sklearn.linear_model import LogisticRegression

log_regr_model = LogisticRegression()
log_regr_model.fit(train_features, train_target)

LogisticRegression()

## 2.3 Исследование качества моделей

In [14]:
valid_features = df_valid.drop(['is_ultra'], axis=1)
valid_target = df_valid['is_ultra']

**Решающее дерево** 
  
Будем исследовать глубину дерева -- параметр *max_depth*. Ограничим максимальную глубину дерева до 100 уровней. Качество модели с заданной глубиной будем оценивать по валидационной выборке

In [15]:
max_depth = 0
accuracy = 0

for i in range(1, 100):
    model = DecisionTreeClassifier(random_state=1456, max_depth=i)
    model.fit(train_features, train_target)
    if accuracy_score(valid_target, model.predict(valid_features)) > accuracy:
        accuracy = accuracy_score(valid_target, model.predict(valid_features))
        max_depth = i
        
print(accuracy)
print(max_depth)

0.8211508553654744
8


**Случайный лес** 
  
Будем исследовать размер леса -- параметр *n_estimators* при уже известной глубине дерева -- *max_depth* = 8. Ограничим максимальный размер леса до 100. Качество модели с заданной глубиной и размером будем оценивать по валидационной выборке

In [16]:
n_estimators = 0
accuracy = 0

for i in range(1, 100):
    model = RandomForestClassifier(random_state=1456, max_depth=8, n_estimators=i)
    model.fit(train_features, train_target)
    if accuracy_score(valid_target, model.predict(valid_features)) > accuracy:
        accuracy = accuracy_score(valid_target, model.predict(valid_features))
        n_estimators = i
        
print(accuracy)
print(n_estimators)

0.833592534992224
14


**Логистическая регрессия** 
  
Модель логистической регрессии настроечных парамтеров не имеет. Проверим качество модели на валидационной выборке

In [17]:
print(accuracy_score(valid_target, log_regr_model.predict(valid_features)))

0.7620528771384136


***Промежуточный итог***  
  
По итогам проведённого исследования моеделй можно сделать следующие выводы:
- для повышения качества работы моделей есть возможность их настройки при помощи определённых параметров;
- наилучшим качеством из полученных моделей обладает модель случайного леса с глубиной равной 8 и размером равным 14.

## 2.4 Проверка настроенных моделей на тестовой выборке

In [18]:
test_features = df_test.drop(['is_ultra'], axis=1)
test_target = df_test['is_ultra']

**Решающее дерево** 

In [19]:
decision_tree_model = DecisionTreeClassifier(random_state=1456, max_depth=8)
decision_tree_model.fit(train_features, train_target)
print(accuracy_score(test_target, decision_tree_model.predict(test_features)))

0.8055987558320373


**Случайный лес**

In [20]:
random_forest_model = RandomForestClassifier(random_state=1456, max_depth=8, n_estimators=14)
random_forest_model.fit(train_features, train_target)
print(accuracy_score(test_target, random_forest_model.predict(test_features)))    

0.807153965785381


**Логистическая регрессия**

In [21]:
print(accuracy_score(test_target, log_regr_model.predict(test_features)))

0.7589424572317263


***Промежуточный итог***  
  
Проверка моделей на тестовой выборке показала, что модели решающего дерева и случайного леса по количеству правильных ответов практически равны. При этом качество всех моделей на тестовой выборке чуть ниже, чем на валидационной.

## 2.5 Проверка моделей на вменяемость

Для проверки полученных моделей на вменяемость сравним их качество (*accuracy*) на тестовой выборке с качеством случайной предиктной последовательности

In [22]:
print(accuracy_score(test_target, shisda_predict(size=len(test_target), span=(0, 1))))

0.5007776049766719


***Промежуточный итог***  
  
Проведённая проверка показала, что качество полученных моделей значительно превосходит качество *shisda*-модели (в том числе и логистическая регрессия, имющая наименьший *accuracy*), что говорит о высокой вероятности вменяемости полученных моделей.

# 3 Вывод

В ходе выполнения работы было сделано следующее.  
  
В начале выполнения работы был произведён предварительный обзор данных, были выделены тренировочная, валидационная и тестовая выборки в соотношении 3:1:1 (0.6:0.2:0.2)  
  
Затем были созданы модел *Решающего дерева*, *Случайного леса* и *Логистической регрессии*, качество которых было исследовано.  
По итогам проведённого исследования моеделй были сделаны следующие выводы:
- для повышения качества работы моделей есть возможность их настройки при помощи определённых параметров;
- наилучшим качеством из полученных моделей обладает модель случайного леса с глубиной равной 8 и размером равным 14.

  
Затем была проведена проверка моделей на тестовой выборке, которая показала, что модели решающего дерева и случайного леса по количеству правильных ответов практически равны. При этом качество всех моделей на тестовой выборке чуть ниже, чем на валидационной.
  
В заключении работы была проведена проверка на вменяемость моделей, которая показала, что качество полученных моделей значительно превосходит качество *shisda*-модели (в том числе и логистическая регрессия, имющая наименьший *accuracy*), что говорит о высокой вероятности вменяемости полученных моделей.  
  
***Таким образом, «Мегалайну» можно предложить полученную модель случайного леса, как рекомендательную систему для пользователей при выборе тарифов «Смарт» или «Ультра»***